In [1]:
import pandas as pd
import matplotlib.pyplot as plt
df=pd.read_csv("weather.csv")
df

,Outlook,Temp,Humidity,Windy,Play
0,Rainy,Hot,High,f,no
1,Rainy,Hot,High,t,no
2,Overcast,Hot,High,f,yes
3,Sunny,Mild,High,f,yes
4,Sunny,Cool,Normal,f,yes
5,Sunny,Cool,Normal,t,no
6,Overcast,Cool,Normal,t,yes
7,Rainy,Mild,High,f,no
8,Rainy,Cool,Normal,f,yes
9,Sunny,Mild,Normal,f,yes


In [2]:
total_yes=df[df['Play']=='yes'].shape[0]
total_no=df[df['Play']=='no'].shape[0]
print(total_yes,total_no)

9 5


In [3]:
from collections import defaultdict
prior_prob_class=defaultdict(list)
dependent_class=df.Play.unique()
print(dependent_class)

['no' 'yes']


In [4]:
for cl in dependent_class:
    total_class_count=df[df["Play"]==cl].shape[0]
    prior_prob_class[cl]=total_class_count/df.shape[0]
print(prior_prob_class)

defaultdict(<class 'list'>, {'no': 0.35714285714285715, 'yes': 0.6428571428571429})


In [5]:
for outlook in df.Outlook.unique():
    df_outlook=df[df["Outlook"]==outlook]
    display(df_outlook)
    

,Outlook,Temp,Humidity,Windy,Play
0,Rainy,Hot,High,f,no
1,Rainy,Hot,High,t,no
7,Rainy,Mild,High,f,no
8,Rainy,Cool,Normal,f,yes
10,Rainy,Mild,Normal,t,yes


,Outlook,Temp,Humidity,Windy,Play
2,Overcast,Hot,High,f,yes
6,Overcast,Cool,Normal,t,yes
11,Overcast,Mild,High,t,yes
12,Overcast,Hot,Normal,f,yes


,Outlook,Temp,Humidity,Windy,Play
3,Sunny,Mild,High,f,yes
4,Sunny,Cool,Normal,f,yes
5,Sunny,Cool,Normal,t,no
9,Sunny,Mild,Normal,f,yes
13,Sunny,Mild,High,t,no


In [6]:
outlook_prob=pd.DataFrame(index=df.Outlook.unique(),columns=["yes","no"])
outlook_prob

,yes,no
Rainy,NaN,NaN
Overcast,NaN,NaN
Sunny,NaN,NaN


In [7]:
for outlook in df.Outlook.unique():
    df_outlook=df[df["Outlook"]==outlook]
    display(df_outlook)
    print("When the outlook is",outlook,'total number of elements',df_outlook.shape[0])
    yes_count=(df_outlook.Play=='yes').sum()
    no_count=(df_outlook.Play=='no').sum()
    print(f"yes_prob:{yes_count/total_yes} and no_prob:{no_count/total_no}")
    
    outlook_prob.loc[outlook,"yes"]=yes_count/total_yes
    outlook_prob.loc[outlook,"no"]=no_count/total_yes
display(outlook_prob)

,Outlook,Temp,Humidity,Windy,Play
0,Rainy,Hot,High,f,no
1,Rainy,Hot,High,t,no
7,Rainy,Mild,High,f,no
8,Rainy,Cool,Normal,f,yes
10,Rainy,Mild,Normal,t,yes


When the outlook is Rainy total number of elements 5
yes_prob:0.2222222222222222 and no_prob:0.6


,Outlook,Temp,Humidity,Windy,Play
2,Overcast,Hot,High,f,yes
6,Overcast,Cool,Normal,t,yes
11,Overcast,Mild,High,t,yes
12,Overcast,Hot,Normal,f,yes


When the outlook is Overcast total number of elements 4
yes_prob:0.4444444444444444 and no_prob:0.0


,Outlook,Temp,Humidity,Windy,Play
3,Sunny,Mild,High,f,yes
4,Sunny,Cool,Normal,f,yes
5,Sunny,Cool,Normal,t,no
9,Sunny,Mild,Normal,f,yes
13,Sunny,Mild,High,t,no


When the outlook is Sunny total number of elements 5
yes_prob:0.3333333333333333 and no_prob:0.4


,yes,no
Rainy,0.222222,0.333333
Overcast,0.444444,0.0
Sunny,0.333333,0.222222


In [8]:
likelihood_margin_dict=defaultdict(list)
features=df.columns[:-1]
print(features)

Index(['Outlook', 'Temp', 'Humidity', 'Windy'], dtype='object')


In [17]:
for feature in features:
    print('feature:',feature)
    feature_df=pd.DataFrame(index=df[feature].unique(),columns=["yes","no"])
    for feature_value in df[feature].unique():
        df_feature_value=df[df[feature]==feature_value]
        yes_count=(df_feature_value.Play=='yes').sum()
        no_count=(df_feature_value.Play=='no').sum()
        yes_prob=yes_count/total_yes
        no_prob=no_count/total_no
        feature_df.loc[feature_value,"yes"]=yes_prob
        feature_df.loc[feature_value,"no"]=no_prob
    display(feature_df)
    likelihood_margin_dict[feature]=feature_df
print(feature_df) 

feature: Play


,yes,no
no,0.0,1.0
yes,1.0,0.0


feature: Windy


,yes,no
f,0.666667,0.4
t,0.333333,0.6


feature: Humidity


...

feature: Temp


,yes,no
Hot,0.222222,0.4
Mild,0.444444,0.4
Cool,0.333333,0.2


feature: Outlook


,yes,no
Rainy,0.222222,0.6
Overcast,0.444444,0.0
Sunny,0.333333,0.4


               yes   no
Rainy     0.222222  0.6
Overcast  0.444444  0.0
Sunny     0.333333  0.4


In [18]:
x=['Rainy',"Mild","Normal","t"]
features=df.columns[::-1]
likelihood_yes=1
likelihood_no=1
marginal=1
for feature_value ,feature in zip(x,features):
    likelihood_yes=likelihood_yes*likelihood_margin_dict[feature].loc[feature_value]['yes']
    likelihood_no=likelihood_no*likelihood_margin_dict[feature].loc[feature_value]['no']

probability_yes=(likelihood_yes* prior_prob_class["yes"])
probability_no=(likelihood_no* prior_prob_class["no"])
print(probability_yes)
print(probability_no)

KeyError: 'Rainy'